## Toxic comment classification
### Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc, f1_score

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse

## Read data set

In [2]:
toxic = pd.read_csv('toxicity_data/train.csv') #there's also a test dataset but it doesn't have labels b/c kaggle.
print('Number of rows and columns in the train data set:',toxic.shape)

#unlabeled data
incel_df = pd.read_csv('new_IncelTears_posts.csv')
slate_df = pd.read_csv('new_slatestarcodex_posts.csv')

#turn multi-class into single class classifier
target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y = toxic[target_col]
y['sum'] = y.sum(axis=1).astype(bool).astype(int)

#splits data, creates holdout dataset
X_train, X_holdout, y_train, y_holdout = train_test_split(toxic, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


Number of rows and columns in the train data set: (159571, 8)


## Text preprocessing - TF-IDF up to trigrams

In [3]:
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
tr_vect = vect_word.fit_transform(X_train['comment_text'])
ts_vect = vect_word.transform(X_test['comment_text'])

incel_vect = vect_word.transform(incel_df['title'])
slate_vect = vect_word.transform(slate_df['title'])

#took 50 seconds on 150k samples

## LR Model

In [4]:
lr = LogisticRegression(C=2,random_state = 42,class_weight = 'balanced')
lr.fit(tr_vect,y_train['sum'])

pred =  lr.predict(ts_vect)
print('\nConfusion matrix\n',confusion_matrix(y_test['sum'],pred))
print(classification_report(y_test['sum'],pred))


Confusion matrix
 [[21668  1211]
 [  428  2225]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     22879
           1       0.65      0.84      0.73      2653

   micro avg       0.94      0.94      0.94     25532
   macro avg       0.81      0.89      0.85     25532
weighted avg       0.95      0.94      0.94     25532



In [5]:
f1_score(y_test['sum'],pred)  

0.7308260798160617

## get word counts

In [6]:
model_coefs = lr.coef_[0,:]
model_keys = np.argsort(model_coefs)
vocab_dict = {v: k for k, v in vect_word.vocabulary_.items()}
top_n_words = 100
pos_words = model_keys[-top_n_words::][::-1]
neg_words = model_keys[:top_n_words]

for p, n in zip(pos_words, neg_words):
    print(f"{vocab_dict[p]: <20} \t\t {vocab_dict[n]: <20}")

fuck                 		 thanks              
fucking              		 best                
shit                 		 talk                
idiot                		 interested          
stupid               		 thank               
ass                  		 stop vandalizing    
bullshit             		 consensus           
asshole              		 article             
bitch                		 ips                 
suck                 		 wikiproject         
cunt                 		 wp                  
crap                 		 mentioned           
moron                		 future              
dick                 		 test                
faggot               		 agree               
sucks                		 help                
idiots               		 official            
penis                		 request             
jerk                 		 appreciate          
hell                 		 dispute             
pathetic             		 title               
bastard              		 disagree            
fucked    